## **Datos Sanidad**

In [51]:
import pandas as pd
import numpy as np
import os
import sys

In [ ]:
municipios_cluster = pd.read_csv(r"external_data/municipios_con_cluster.csv", sep=",")

### **farmacias_100**

In [53]:
import os
import re
import unicodedata
import numpy as np
import pandas as pd

# -----------------------------
# CONFIG
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"     # largo: Municipios;Sexo;Periodo;Total
FARMACIAS_CSV = "data/sanidad/farmacias.csv"              # ancho: Nombre;...;2022;2023;...

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE
    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"
    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# ============================================================
# 1) POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)
pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) FARMACIAS (ancho)
# ============================================================
far = pd.read_csv(FARMACIAS_CSV, sep=";", dtype=str)  # cambia sep si tu CSV es ","
far["Nombre"] = far["Nombre"].apply(normalize_nombre)
far["key"] = far["Nombre"].apply(canon_key)

year_cols_far = [c for c in far.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_far:
    far[c] = far[c].apply(parse_es_number)

farmacias_total = far[["key", "Nombre"] + year_cols_far].copy()

# ============================================================
# 3) CLUSTERS (ya cargado)
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR: intersección y >= 2022
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_far = {int(c) for c in year_cols_far}

years_to_process = sorted(y for y in years_pobl.intersection(years_far) if y >= 2022)
print("Años a procesar (>=2022):", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    numero_farmacias = farmacias_total[["key", "Nombre", col]].rename(columns={col: "farmacias_total"})
    poblacion_anio = total_poblacion[["key", col]].rename(columns={col: "total_poblacion"})

    datos = pd.merge(numero_farmacias, poblacion_anio, on="key", how="inner")

    # Añadir cluster + nombre final desde cluster si existe
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"], errors="ignore")

    datos["farmacias_total"] = pd.to_numeric(datos["farmacias_total"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # Farmacias por 100 habitantes
    datos["farmacias_100"] = (datos["farmacias_total"] / datos["total_poblacion"]) * 100
    datos["farmacias_100"] = datos["farmacias_100"].replace([np.inf, -np.inf], np.nan).fillna(0)

    # más es mejor
    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["farmacias_100"].mean()
        desviacion = grupo["farmacias_100"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)
        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "farmacias_100"}
    )

    output_path = f"data_interfaz/sanidad/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}farmacias_100.csv", index=False)
    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (>=2022): [2023, 2024]
Archivo del 2023 exportado a data_interfaz/sanidad/normalizacion_final/2023/
Archivo del 2024 exportado a data_interfaz/sanidad/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_28436\1823899455.py:150: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_28436\1823899455.py:150: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_

### **centrosSociales_10**

In [54]:
centrosSociales_total = pd.read_csv("data/sanidad/centros_sociales.csv", sep=";")

In [55]:
import os
import re
import unicodedata
import numpy as np
import pandas as pd

# -----------------------------
# Helpers (igual que antes)
# -----------------------------
def parse_es_number(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE si viene
    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"
    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

def preparar_df_ancho(df: pd.DataFrame):
    df = df.copy()
    df["Nombre"] = df["Nombre"].apply(normalize_nombre)
    df["key"] = df["Nombre"].apply(canon_key)
    year_cols = [c for c in df.columns if re.fullmatch(r"\d{4}", str(c))]
    for c in year_cols:
        df[c] = df[c].apply(parse_es_number)
    return df, year_cols

# ============================================================
# 0) PREPARAR DF CENTROS SOCIALES (ancho) + CLUSTERS
#    NOTA: total_poblacion YA debe existir pivotado con "key"
# ============================================================
centrosSociales_total, year_cols_cs = preparar_df_ancho(centrosSociales_total)

municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)
municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 1) AÑOS A PROCESAR: intersección con población y >= 2022
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_cs = {int(c) for c in year_cols_cs}

years_to_process = sorted(y for y in years_pobl.intersection(years_cs) if y >= 2022)
print("Años a procesar (>=2022):", years_to_process)

# ============================================================
# 2) FUNCIÓN POR AÑO
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    numero_cs = centrosSociales_total[["key", "Nombre", col]].rename(columns={col: "centrosSociales_total"})
    poblacion_anio = total_poblacion[["key", col]].rename(columns={col: "total_poblacion"})

    datos = pd.merge(numero_cs, poblacion_anio, on="key", how="inner")

    # Añadir cluster + nombre final desde cluster si existe
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"], errors="ignore")

    datos["centrosSociales_total"] = pd.to_numeric(datos["centrosSociales_total"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # Centros sociales por 10 habitantes (según tu fórmula)
    datos["centrosSociales_10"] = (datos["centrosSociales_total"] / datos["total_poblacion"]) * 10
    datos["centrosSociales_10"] = datos["centrosSociales_10"].replace([np.inf, -np.inf], np.nan).fillna(0)

    datos["type"] = 1  # más es mejor

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["centrosSociales_10"].mean()
        desviacion = grupo["centrosSociales_10"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)
        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "centrosSociales_10"}
    )

    output_path = f"data_interfaz/sanidad/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}centrosSociales_10.csv", index=False)
    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 3) PROCESAR
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (>=2022): [2023, 2024, 2025]
Archivo del 2023 exportado a data_interfaz/sanidad/normalizacion_final/2023/
Archivo del 2024 exportado a data_interfaz/sanidad/normalizacion_final/2024/
Archivo del 2025 exportado a data_interfaz/sanidad/normalizacion_final/2025/


C:\Users\aitan\AppData\Local\Temp\ipykernel_28436\158877729.py:123: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_28436\158877729.py:123: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_at

### **centros_100**

In [56]:
centros_total = pd.read_csv("data/sanidad/centros_salud.csv", sep=";")

In [57]:
import os
import re
import unicodedata
import numpy as np
import pandas as pd

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE si viene
    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"
    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

def preparar_df_ancho(df: pd.DataFrame):
    df = df.copy()
    df["Nombre"] = df["Nombre"].apply(normalize_nombre)
    df["key"] = df["Nombre"].apply(canon_key)
    year_cols = [c for c in df.columns if re.fullmatch(r"\d{4}", str(c))]
    for c in year_cols:
        df[c] = df[c].apply(parse_es_number)
    return df, year_cols

# ============================================================
# 0) PREPARAR DF CENTROS (ancho) + CLUSTERS
#    NOTA: total_poblacion ya debe existir pivotado con "key"
# ============================================================
centros_total, year_cols_cent = preparar_df_ancho(centros_total)

municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)
municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 1) AÑOS A PROCESAR: intersección con población y >= 2022
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_cent = {int(c) for c in year_cols_cent}

years_to_process = sorted(y for y in years_pobl.intersection(years_cent) if y >= 2022)
print("Años a procesar (>=2022):", years_to_process)

# ============================================================
# 2) FUNCIÓN POR AÑO
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    numero_centros = centros_total[["key", "Nombre", col]].rename(columns={col: "centros_total"})
    poblacion_anio = total_poblacion[["key", col]].rename(columns={col: "total_poblacion"})

    datos = pd.merge(numero_centros, poblacion_anio, on="key", how="inner")

    # Añadir cluster + nombre final desde cluster si existe
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"], errors="ignore")

    datos["centros_total"] = pd.to_numeric(datos["centros_total"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # OJO: tu fórmula multiplica por 10 (no por 100). Mantengo tu lógica.
    datos["centros_100"] = (datos["centros_total"] / datos["total_poblacion"]) * 10
    datos["centros_100"] = datos["centros_100"].replace([np.inf, -np.inf], np.nan).fillna(0)

    datos["type"] = 1  # más es mejor

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["centros_100"].mean()
        desviacion = grupo["centros_100"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)
        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "centros_100"}
    )

    output_path = f"data_interfaz/sanidad/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}centros_100.csv", index=False)
    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 3) PROCESAR
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (>=2022): [2023, 2024]
Archivo del 2023 exportado a data_interfaz/sanidad/normalizacion_final/2023/
Archivo del 2024 exportado a data_interfaz/sanidad/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_28436\1721774206.py:123: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_28436\1721774206.py:123: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_

# *Selección de cuatro (4) especialidades*

Bajo la condición de que al menos 100 municipios la oferten

In [58]:
import pandas as pd

# Cargar el CSV
df = pd.read_csv("data/sanidad/especialidades_centros.csv", sep=';', encoding='ISO-8859-1')

# Agrupar por tipo de centro y obtener lista única de municipios (ignorando NaNs)
df_agrupado = df.groupby('centro_tipo')['municipio_nombre'].apply(
    lambda x: sorted(set(filter(pd.notna, x)))
).reset_index()

# Filtrar los tipos de centro con 100 o más municipios únicos
centros_mas_100_municipios = df_agrupado[df_agrupado['municipio_nombre'].apply(len) >= 100]

# Mostrar solo los tipos de centro
print(centros_mas_100_municipios['centro_tipo'].tolist())

['Clínica dental', 'Consulta de otros profesionales sanitarios', 'Consultorio de atención primaria', 'Servicio sanitario integrado en una organización no sanitaria']


In [59]:
import pandas as pd
import os

# Rutas de los archivos
ruta_especialidades = "data/sanidad/especialidades_centros.csv"
ruta_municipios = "external_data/lau_id_nombre.csv"
output_csv = "external_data/sanidad/especialidades_sanidad_con_municipio.csv"

# Cargar los archivos
df_centros = pd.read_csv(ruta_especialidades, sep=';',encoding='ISO-8859-1')
df_municipios = pd.read_csv(ruta_municipios)

# Asegurarse de que los códigos postales sean string, eliminando decimales, manejando NaN
df_centros['direccion_codigo_postal'] = (
    df_centros['direccion_codigo_postal']
    .apply(lambda x: str(int(x)) if pd.notnull(x) else pd.NA)
)

df_municipios['id'] = df_municipios['id'].astype(str).str.strip()

# Hacemos el merge para añadir la columna con el nombre del municipio
df_centros = df_centros.merge(
    df_municipios.rename(columns={'id': 'direccion_codigo_postal', 'Nombre': 'municipio_nombre_oficial'}),
    on='direccion_codigo_postal',
    how='left'
)

# Crear carpeta si aplica
out_dir = os.path.dirname(output_csv)
if out_dir:  # solo si no es ''
    os.makedirs(out_dir, exist_ok=True)

# Guardar
df_centros.to_csv(output_csv, index=False, encoding="utf-8", sep=";")
print(f"✅ Archivo generado: {output_csv}")

✅ Archivo generado: external_data/sanidad/especialidades_sanidad_con_municipio.csv


In [60]:
import pandas as pd
import os

# Rutas de los archivos
ruta_centros = "data/sanidad/especialidades_centros.csv"
ruta_municipios = "external_data/lau_id_nombre.csv"
output_dir = "data/sanidad/especialidades_seleccionadas"

# Cargar los archivos
df_centros = pd.read_csv(ruta_especialidades, sep=';',encoding='ISO-8859-1')
df_municipios = pd.read_csv(ruta_municipios)

# Normalizamos los nombres
df_centros['municipio_nombre'] = df_centros['municipio_nombre'].str.strip().str.lower()
df_municipios['nombre'] = df_municipios['nombre'].str.strip().str.lower()

# Eliminamos duplicados en municipios por precaución
df_municipios = df_municipios.drop_duplicates(subset='id')

# Lista de tipos de centro a procesar
tipos_filtrados = [
    'Clínica dental',
    'Consulta de otros profesionales sanitarios',
    'Consultorio de atención primaria',
    'Servicio sanitario integrado en una organización no sanitaria'
]

# Crear carpeta de salida
os.makedirs(output_dir, exist_ok=True)

# Procesar cada tipo
for tipo in tipos_filtrados:
    df_filtrado = df_centros[df_centros['centro_tipo'] == tipo]

    municipios_con_tipo = df_filtrado['municipio_nombre'].nunique()

    if municipios_con_tipo >= 100:
        print(f"Procesando: {tipo} ({municipios_con_tipo} municipios)")

        # Conteo de centros por municipio
        conteo = df_filtrado.groupby('municipio_nombre').size().reset_index(name='2024')

        # Merge con municipios oficiales por nombre
        df_final = df_municipios.merge(
            conteo,
            left_on='nombre',
            right_on='municipio_nombre',
            how='left'
        ).drop(columns=['municipio_nombre'])

        df_final['2024'] = df_final['2024'].fillna(0).astype(int)
        df_final['nombre'] = df_final['nombre'].apply(lambda x: x.title())

        # Reordenar columnas y añadir Serie
        df_final.insert(0, 'Serie', 'Municipio')
        df_final = df_final[['Serie', 'id', 'nombre', '2024']]
        df_final = df_final.rename(columns={'nombre': 'Nombre'})

        # Guardar archivo de salida
        nombre_archivo = tipo.lower().replace(' ', '_').replace('ñ', 'n') + ".csv"
        ruta_salida = os.path.join(output_dir, nombre_archivo)
        df_final.to_csv(ruta_salida, index=False, sep=';')

        print(f"✅ Generado: {ruta_salida}")

    else:
        print(f"⚠️ Se omite {tipo}, solo en {municipios_con_tipo} municipios")

Procesando: Clínica dental (113 municipios)
✅ Generado: data/sanidad/especialidades_seleccionadas\clínica_dental.csv
Procesando: Consulta de otros profesionales sanitarios (107 municipios)
✅ Generado: data/sanidad/especialidades_seleccionadas\consulta_de_otros_profesionales_sanitarios.csv
Procesando: Consultorio de atención primaria (129 municipios)
✅ Generado: data/sanidad/especialidades_seleccionadas\consultorio_de_atención_primaria.csv
Procesando: Servicio sanitario integrado en una organización no sanitaria (123 municipios)
✅ Generado: data/sanidad/especialidades_seleccionadas\servicio_sanitario_integrado_en_una_organización_no_sanitaria.csv


### **clinicaDental_10**

In [ ]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"          # sep=";"
CLINICADENTAL_CSV = "data/sanidad/especialidades_seleccionadas/clínica_dental.csv"     # sep=";"  <-- ajusta ruta/nombre

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, etc."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # deja solo alfanumérico
    return s

# ============================================================
# 1) CARGA Y PREPARACIÓN POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)

# Filtrar Sexo Total
pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

# Normalizar y parsear
pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

# Pivot a formato ancho
total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
# Columnas a string (años)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN CLÍNICAS DENTALES (ya ancho)
# ============================================================
cd = pd.read_csv(CLINICADENTAL_CSV, sep=";", dtype=str)

cd["Nombre"] = cd["Nombre"].apply(normalize_nombre)
cd["key"] = cd["Nombre"].apply(canon_key)

# Detectar columnas año
year_cols = [c for c in cd.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    cd[c] = cd[c].apply(parse_es_number)

clinicadental_total = cd[["key", "Nombre"] + year_cols].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
#     Debe tener columnas: Nombre, Cluster (y otras)
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección población vs clínicas)
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_cd = {int(c) for c in year_cols}

# Si quieres SOLO >= 2022:
years_to_process = sorted(y for y in years_pobl.intersection(years_cd) if y >= 2022)
# Si NO quieres filtro, usa:
# years_to_process = sorted(years_pobl.intersection(years_cd))

print("Años a procesar:", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    # Selección por año
    numero_cdental = (
        clinicadental_total[["key", "Nombre", col]]
        .rename(columns={col: "clinicadental_total"})
    )

    poblacion_anio = (
        total_poblacion[["key", col]]
        .rename(columns={col: "total_poblacion"})
    )

    # Merge clínicas + población por key
    datos = pd.merge(numero_cdental, poblacion_anio, on="key", how="inner")

    # Añadir cluster (y si quieres usar el nombre del cluster como salida)
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    # Usar Nombre del cluster si existe; si no, mantener el de clínicas
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    # Numéricos
    datos["clinicadental_total"] = pd.to_numeric(datos["clinicadental_total"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # Clínicas dentales por X habitantes
    # (tu fórmula original multiplicaba por 10; la mantengo)
    datos["clinicaDental_10"] = ((datos["clinicadental_total"] / datos["total_poblacion"]) * 10).fillna(0)

    # Más es mejor
    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["clinicaDental_10"].mean()
        desviacion = grupo["clinicaDental_10"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    # Con/sin cluster
    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    # Salida final
    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "clinicaDental_10"}
    )

    # Guardar
    output_path = f"data_interfaz/sanidad/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}clinicaDental_10.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR TODOS LOS AÑOS
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar: [2024]
Archivo del 2024 exportado a data_interfaz/sanidad/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_28436\4071580087.py:183: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)


### **otroConsulta_100**

In [ ]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"          # sep=";"
OTROCONSULTA_CSV = "data/sanidad/especialidades_seleccionadas/consulta_de_otros_profesionales_sanitarios.csv"          # sep=";"  <-- ajusta ruta/nombre

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, etc."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # deja solo alfanumérico
    return s

# ============================================================
# 1) CARGA Y PREPARACIÓN POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)

# Filtrar Sexo Total
pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

# Normalizar y parsear
pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

# Pivot a formato ancho
total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN OTROCONSULTA (ya ancho)
# ============================================================
ot = pd.read_csv(OTROCONSULTA_CSV, sep=";", dtype=str)

ot["Nombre"] = ot["Nombre"].apply(normalize_nombre)
ot["key"] = ot["Nombre"].apply(canon_key)

# Detectar columnas año
year_cols = [c for c in ot.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    ot[c] = ot[c].apply(parse_es_number)

otroconsulta_total = ot[["key", "Nombre"] + year_cols].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección población vs otroconsulta) >= 2022
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_ot = {int(c) for c in year_cols}

years_to_process = sorted(y for y in years_pobl.intersection(years_ot) if y >= 2022)
print("Años a procesar:", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    numero_ot = (
        otroconsulta_total[["key", "Nombre", col]]
        .rename(columns={col: "otroconsulta_total"})
    )

    poblacion_anio = (
        total_poblacion[["key", col]]
        .rename(columns={col: "total_poblacion"})
    )

    # Merge indicador + población por key
    datos = pd.merge(numero_ot, poblacion_anio, on="key", how="inner")

    # Añadir cluster
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    # Usar Nombre del cluster si existe; si no, mantener el del indicador
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    # Numéricos
    datos["otroconsulta_total"] = pd.to_numeric(datos["otroconsulta_total"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # Indicador por 100 habitantes
    datos["otroConsulta_100"] = ((datos["otroconsulta_total"] / datos["total_poblacion"]) * 100).fillna(0)

    datos["type"] = 1  # más es mejor

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["otroConsulta_100"].mean()
        desviacion = grupo["otroConsulta_100"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    # Salida final
    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "otroConsulta_100"}
    )

    # Guardar
    output_path = f"data_interfaz/sanidad/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}otroConsulta_100.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR TODOS LOS AÑOS
# ============================================================
for y in years_to_process:
    procesar_anio(y)

# Si SOLO quieres 2023:
# procesar_anio(2023)


Años a procesar: [2024]
Archivo del 2024 exportado a data_interfaz/sanidad/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_28436\1622194713.py:173: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)


### **consultaPrimaria_100**

In [ ]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"              # sep=";"
CONSULTA_PRIMARIA_CSV = "data/sanidad/especialidades_seleccionadas/consultorio_de_atención_primaria.csv"     # sep=";"  <-- ajusta ruta/nombre

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, etc."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # deja solo alfanumérico
    return s

# ============================================================
# 1) CARGA Y PREPARACIÓN POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)

pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN CONSULTA PRIMARIA (ya ancho)
# ============================================================
cp = pd.read_csv(CONSULTA_PRIMARIA_CSV, sep=";", dtype=str)

cp["Nombre"] = cp["Nombre"].apply(normalize_nombre)
cp["key"] = cp["Nombre"].apply(canon_key)

year_cols = [c for c in cp.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    cp[c] = cp[c].apply(parse_es_number)

consultaprimaria_total = cp[["key", "Nombre"] + year_cols].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección población vs consulta primaria) >= 2022
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_cp = {int(c) for c in year_cols}

years_to_process = sorted(y for y in years_pobl.intersection(years_cp) if y >= 2022)
print("Años a procesar:", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    numero_cp = (
        consultaprimaria_total[["key", "Nombre", col]]
        .rename(columns={col: "consultaprimaria_total"})
    )

    poblacion_anio = (
        total_poblacion[["key", col]]
        .rename(columns={col: "total_poblacion"})
    )

    datos = pd.merge(numero_cp, poblacion_anio, on="key", how="inner")

    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    datos["consultaprimaria_total"] = pd.to_numeric(datos["consultaprimaria_total"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # Mantengo tu fórmula *10
    datos["consultaPrimaria_100"] = ((datos["consultaprimaria_total"] / datos["total_poblacion"]) * 10).fillna(0)

    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["consultaPrimaria_100"].mean()
        desviacion = grupo["consultaPrimaria_100"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "consultaPrimaria_100"}
    )

    output_path = f"data_interfaz/sanidad/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}consultaPrimaria_100.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR (si SOLO quieres 2023, deja esta línea)
# ============================================================
# procesar_anio(2023)

# Si quieres todos los años disponibles >=2022:
for y in years_to_process:
    procesar_anio(y)


Años a procesar: [2024]
Archivo del 2024 exportado a data_interfaz/sanidad/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_28436\3132581442.py:165: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)


### **orgNoSanitaria_100**

In [ ]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
POBLACION_CSV = "external_data/poblacion_total.csv"              # sep=";"
ORG_NO_SANITARIA_CSV = "data/sanidad/especialidades_seleccionadas/servicio_sanitario_integrado_en_una_organización_no_sanitaria.csv"       # sep=";"  <-- ajusta ruta/nombre

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, etc."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)  # deja solo alfanumérico
    return s

# ============================================================
# 1) CARGA Y PREPARACIÓN POBLACIÓN (largo -> ancho)
# ============================================================
pobl = pd.read_csv(POBLACION_CSV, sep=";", dtype=str)

pobl["Sexo"] = pobl["Sexo"].astype(str).str.strip()
pobl = pobl[pobl["Sexo"] == "Total"].copy()

pobl["Nombre"] = pobl["Municipios"].apply(normalize_nombre)
pobl["key"] = pobl["Nombre"].apply(canon_key)
pobl["Periodo"] = pd.to_numeric(pobl["Periodo"], errors="coerce").astype("Int64")
pobl["Total"] = pobl["Total"].apply(parse_es_number)

total_poblacion = (
    pobl.pivot_table(index=["key", "Nombre"], columns="Periodo", values="Total", aggfunc="first")
    .reset_index()
)
total_poblacion.columns = ["key", "Nombre"] + [str(c) for c in total_poblacion.columns[2:]]

# ============================================================
# 2) CARGA Y PREPARACIÓN ORG NO SANITARIA (ya ancho)
# ============================================================
org = pd.read_csv(ORG_NO_SANITARIA_CSV, sep=";", dtype=str)

org["Nombre"] = org["Nombre"].apply(normalize_nombre)
org["key"] = org["Nombre"].apply(canon_key)

year_cols = [c for c in org.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols:
    org[c] = org[c].apply(parse_es_number)

orgNoSanitaria_total = org[["key", "Nombre"] + year_cols].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección población vs org no sanitaria) >= 2022
# ============================================================
years_pobl = {int(c) for c in total_poblacion.columns if re.fullmatch(r"\d{4}", str(c))}
years_org = {int(c) for c in year_cols}

years_to_process = sorted(y for y in years_pobl.intersection(years_org) if y >= 2022)
print("Años a procesar:", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    numero_org = (
        orgNoSanitaria_total[["key", "Nombre", col]]
        .rename(columns={col: "orgNoSanitaria_total"})
    )

    poblacion_anio = (
        total_poblacion[["key", col]]
        .rename(columns={col: "total_poblacion"})
    )

    datos = pd.merge(numero_org, poblacion_anio, on="key", how="inner")

    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    datos["orgNoSanitaria_total"] = pd.to_numeric(datos["orgNoSanitaria_total"], errors="coerce")
    datos["total_poblacion"] = pd.to_numeric(datos["total_poblacion"], errors="coerce")

    # Mantengo tu fórmula *10
    datos["orgNoSanitaria_total_100"] = ((datos["orgNoSanitaria_total"] / datos["total_poblacion"]) * 10).fillna(0)

    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["orgNoSanitaria_total_100"].mean()
        desviacion = grupo["orgNoSanitaria_total_100"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "orgNoSanitaria_total_100"}
    )

    output_path = f"data_interfaz/sanidad/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}orgNoSanitaria_total_100.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR (si SOLO quieres 2023, deja esta línea)
# ============================================================
#procesar_anio(2023)

# Si quieres todos los años disponibles >=2022:
for y in years_to_process:
    procesar_anio(y)


Años a procesar: [2024]
Archivo del 2024 exportado a data_interfaz/sanidad/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_28436\2817288964.py:165: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
